In [1]:
# 1. 데이터 로딩
import pandas as pd

# titanic 데이터셋 파일 경로 (파일명을 실제 경로로 변경하세요)
file_path = "titanic.csv"
df = pd.read_csv(file_path)

# 데이터 미리보기
print("데이터 미리보기:")
print(df.head())


데이터 미리보기:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [3]:
# 2. 결측치 확인
print("\n결측치 확인 (컬럼별 결측치 개수):")
print(df.isnull().sum())



결측치 확인 (컬럼별 결측치 개수):
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [7]:
# 3. 결측치 처리 - Age 컬럼 결측치는 평균 나이로 채우기
if 'Age' in df.columns:
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    

# 결측치 처리 후 다시 확인
print("\n결측치 처리 후 (Age 결측치 평균 대체):")
print(df.isnull().sum())



결측치 처리 후 (Age 결측치 평균 대체):
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [9]:
# 4. 레이블 확인 (imbalanced data)
# 과제에서는 df['label']을 사용한다고 했으므로, 만약 타이타닉 데이터셋의 타겟이 'Survived'라면 이름을 변경합니다.
if 'Survived' in df.columns:
    df.rename(columns={'Survived': 'label'}, inplace=True)

print("\n레이블 분포 (변환 전):")
print(df['label'].value_counts())



레이블 분포 (변환 전):
label
0    549
1    342
Name: count, dtype: int64


In [11]:
# 5. 불필요한 컬럼 제거
# 일반적으로 타이타닉 데이터에서 분석에 크게 기여하지 않는 컬럼들을 제거합니다.
# 예시: PassengerId, Name, Ticket, Cabin
cols_to_drop = []
for col in ['PassengerId', 'Name', 'Ticket', 'Cabin']:
    if col in df.columns:
        cols_to_drop.append(col)
        
df.drop(columns=cols_to_drop, inplace=True)

print("\n제거 후 컬럼 리스트:")
print(df.columns)


제거 후 컬럼 리스트:
Index(['label', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


In [13]:
# 6. 엔코딩 (숫자 변환)
# 범주형 변수(예: Sex, Embarked 등)를 더미 변수로 변환합니다.
# 변환 전 레이블 분포도 확인
print("\n변환 전 'label' 분포:")
print(df['label'].value_counts())

# 범주형 변수 인코딩: 만약 Sex, Embarked 등 존재하면 get_dummies를 사용
categorical_cols = df.select_dtypes(include=['object']).columns
print("\n인코딩 전 범주형 변수:")
print(categorical_cols)

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# 변환 후 데이터 미리보기
print("\n데이터 변환 후 미리보기:")
print(df.head())



변환 전 'label' 분포:
label
0    549
1    342
Name: count, dtype: int64

인코딩 전 범주형 변수:
Index(['Sex', 'Embarked'], dtype='object')

데이터 변환 후 미리보기:
   label  Pclass   Age  SibSp  Parch     Fare  Sex_male  Embarked_Q  \
0      0       3  22.0      1      0   7.2500      True       False   
1      1       1  38.0      1      0  71.2833     False       False   
2      1       3  26.0      0      0   7.9250     False       False   
3      1       1  35.0      1      0  53.1000     False       False   
4      0       3  35.0      0      0   8.0500      True       False   

   Embarked_S  
0        True  
1       False  
2        True  
3        True  
4        True  


In [15]:
# 7. 5가지 분류 모델 (RF, DT, LR, KNN, SVM) 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# X: feature, y: label
X = df.drop('label', axis=1)
y = df['label']

# 학습/테스트 데이터 분할 (예: 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 각 모델별 성능을 저장할 리스트
results = {}

# (1) Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
results['Random Forest'] = {
    "accuracy": accuracy_score(y_test, y_pred),
    "confusion_matrix": confusion_matrix(y_test, y_pred)
}

# (2) Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
results['Decision Tree'] = {
    "accuracy": accuracy_score(y_test, y_pred),
    "confusion_matrix": confusion_matrix(y_test, y_pred)
}

# (3) Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=200, random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
results['Logistic Regression'] = {
    "accuracy": accuracy_score(y_test, y_pred),
    "confusion_matrix": confusion_matrix(y_test, y_pred)
}

# (4) K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
results['KNN'] = {
    "accuracy": accuracy_score(y_test, y_pred),
    "confusion_matrix": confusion_matrix(y_test, y_pred)
}

# (5) Support Vector Machine
from sklearn.svm import SVC
svm = SVC(random_state=42)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
results['SVM'] = {
    "accuracy": accuracy_score(y_test, y_pred),
    "confusion_matrix": confusion_matrix(y_test, y_pred)
}

# 결과 출력
for model_name, metrics in results.items():
    print(f"\n모델: {model_name}")
    print("정확도:", metrics["accuracy"])
    print("Confusion Matrix:")
    print(metrics["confusion_matrix"])



모델: Random Forest
정확도: 0.8044692737430168
Confusion Matrix:
[[90 15]
 [20 54]]

모델: Decision Tree
정확도: 0.7932960893854749
Confusion Matrix:
[[88 17]
 [20 54]]

모델: Logistic Regression
정확도: 0.8100558659217877
Confusion Matrix:
[[90 15]
 [19 55]]

모델: KNN
정확도: 0.7039106145251397
Confusion Matrix:
[[86 19]
 [34 40]]

모델: SVM
정확도: 0.659217877094972
Confusion Matrix:
[[99  6]
 [55 19]]


C:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
